# BentoML을 활용한 모델 서빙
- 참고: 
    - https://docs.bentoml.org/en/latest/quickstart.html
    - https://github.com/bentoml/gallery

In [ ]:
!pip install bentoml

## Scikit Learn으로 간단하게 테스트

In [ ]:
from sklearn import svm
from sklearn import datasets

# Load training data
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Model Training
clf = svm.SVC(gamma='scale')
clf.fit(X, y)

### BentoML 프레임워크에 맞게 서빙코드를 작성

In [ ]:
%%writefile iris_classifier.py
import pandas as pd

from bentoml import env, artifacts, api, BentoService
from bentoml.adapters import DataframeInput
from bentoml.frameworks.sklearn import SklearnModelArtifact

@env(infer_pip_packages=True)
@artifacts([SklearnModelArtifact('model')])
class IrisClassifier(BentoService):
    """
    이 예제는 전처리가 필요없는 간단한 데이터와 간단한 모델이므로 코드가 간결합니다.
    """

    @api(input=DataframeInput(), batch=True)
    def predict(self, df: pd.DataFrame):
        """
        모델의 input에 맞게 ([0.0,0.0,0.0,0.0]) HTTP 요청을 Post로 받았을시 동작하는 코드입니다.
        요청은 http://HOST:PORT/predict로 보냅니다.
        """
        return self.artifacts.model.predict(df)

### BentoML형식으로 훈련된 모델을 서비스로 저장

In [ ]:
#위에서 작성한 IrisClassifier를 import 해주고
from iris_classifier import IrisClassifier

# 인스턴스화를 해준 다음
iris_classifier_service = IrisClassifier()

# 이미 훈련한 모델과 함께 패키징을 합니다.
iris_classifier_service.pack('model', clf)

# 마지막으로 서비스 자체를 저장합니다.
saved_path = iris_classifier_service.save()

### 터미널을 열어 아래 명령어 수행

In [ ]:

#bentoml serve IrisClassifier:latest


### python request로 확인

In [ ]:
import requests
response = requests.post("http://127.0.0.1:5000/predict", json=[[5.1, 3.5, 1.4, 0.2]])
print(response.text)


## Pytorch 예제를 통한 BentoML 사용

wandb 예제에서 사용했던 cifar10 데이터 셋을 위한 모델로 테스트해보겠습니다.

### 전체적인 훈련과정을 진행
이 부분에 대한 설명은 생략하겠습니다.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()

In [ ]:
import torch.optim as optim


net = Net()


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

### 위의 모델을 inference 할 BentoML 프레임워크 코드를 완성

In [ ]:
%%writefile preprocessing.py
from torchvision import transforms


def transform(image):

    my_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    return my_transforms(image)

In [ ]:
%%writefile pytorch_image_classifier.py

from typing import List, BinaryIO

from PIL import Image
import torch
from torch.autograd import Variable
from torchvision import transforms

import bentoml
from bentoml.frameworks.pytorch import PytorchModelArtifact
from bentoml.artifact import PickleArtifact

from bentoml.adapters import FileInput

from preprocessing import transform



@bentoml.env(pip_packages=['torch', 'numpy', 'torchvision', 'scikit-learn'])
@bentoml.artifacts([PytorchModelArtifact('net'), PickleArtifact('classes')])
class PytorchImageClassifier(bentoml.BentoService):
    
     
    #이번에는 input을 파일 자체로 입력받겠습니다.
    #마찬가지로 주소는 HOST:PORT/predict 입니다.
    @bentoml.api(input=FileInput(), batch=True)
    def predict(self, file_streams: List[BinaryIO]) -> List[str]:
        input_datas = []
        # 여러 이미지를 받을 경우를 고려한 반복문
        for fs in file_streams:
            img = Image.open(fs).resize((32, 32))
            input_datas.append(transform(img))

        classes = self.artifacts.classes
        print(classes)
        outputs = self.artifacts.net(Variable(torch.stack(input_datas)))
        _, output_classes = outputs.max(dim=1)

        return [classes[output_class] for output_class in output_classes]

### 훈련된 모델을 포함시키고 서비스로 저장

In [ ]:

from pytorch_image_classifier import PytorchImageClassifier

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

bento_svc = PytorchImageClassifier()
bento_svc.pack('net', net)

bento_svc.pack('classes', classes)


saved_path = bento_svc.save()
print(saved_path)

### 터미널에서 다음 명령어 입력

In [ ]:
#bentoml list -o wide
#bentoml serve PytorchImageClassifier:latest

### 서빙확인

In [ ]:
from IPython.display import Image
Image(filename='data/bird10.png') 

In [ ]:
import requests

file = open('data/bird10.png', 'rb')
upload = {'file':file}
res = requests.post('http://127.0.0.1:5000/predict', files = upload)

print(res.text)
